In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image
from sklearn.metrics import classification_report

import random
import numpy as np
import torch

# Set random seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # If you are using CUDA
torch.backends.cudnn.deterministic = True  # For deterministic results
torch.backends.cudnn.benchmark = False  # For consistency across different environments

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

IMAGE_DIR = 'D:\\PAD-UFES\\images'  
METADATA_PATH = 'D:\\PAD-UFES\\metadata.csv'

metadata = pd.read_csv(METADATA_PATH)

def preprocess_metadata(metadata):
    metadata = metadata.fillna('UNK')

    boolean_cols = [
        'smoke',
        'drink',
        'pesticide',
        'skin_cancer_history',
        'cancer_history',
        'has_piped_water',
        'has_sewage_system',
        'itch',
        'grew',
        'hurt',
        'changed',
        'bleed',
        'elevation',
        'biopsed',
    ]
    # Ensure columns are strings and lowercase
    for col in boolean_cols:
        metadata[col] = metadata[col].astype(str).str.lower()
    
    # Map boolean columns to 1/0/-1
    boolean_mapping = {'true': 1, 'false': 0, 'unk': -1}
    for col in boolean_cols:
        metadata[col] = metadata[col].map(boolean_mapping)
    
    # Handle categorical variables
    categorical_cols = [
        'background_father',
        'background_mother',
        'gender',
        'region',
        'diagnostic',
    ]
    # Convert categorical columns to string
    for col in categorical_cols:
        metadata[col] = metadata[col].astype(str)
    
    # One-hot encode categorical variables
    metadata_encoded = pd.get_dummies(metadata[categorical_cols])
    
    # Normalize numerical variables
    numerical_cols = ['age', 'fitspatrick', 'diameter_1', 'diameter_2']
    # Ensure numerical columns are numeric
    for col in numerical_cols:
        metadata[col] = pd.to_numeric(metadata[col], errors='coerce')
    # Fill NaNs in numerical columns with the mean
    metadata[numerical_cols] = metadata[numerical_cols].fillna(metadata[numerical_cols].mean())
    # Scale numerical columns
    scaler = StandardScaler()
    metadata_numeric = metadata[numerical_cols]
    metadata_numeric_scaled = pd.DataFrame(
        scaler.fit_transform(metadata_numeric), columns=numerical_cols
    )
    
    # Combine all metadata features
    metadata_processed = pd.concat(
        [metadata_numeric_scaled.reset_index(drop=True),
         metadata_encoded.reset_index(drop=True),
         metadata[boolean_cols].reset_index(drop=True)], axis=1
    )
    
    return metadata_processed

# Preprocess metadata
metadata_processed = preprocess_metadata(metadata)

def get_image_paths(metadata, image_dir):
    image_paths = []
    for idx, row in metadata.iterrows():
        filename = row['img_id']
        # Ensure filename is a string
        filename = str(filename)
        # Check if filename has an extension
        if not filename.endswith(('.jpg', '.jpeg', '.png')):
            # Try common extensions
            possible_extensions = ['.jpg', '.jpeg', '.png']
            found = False
            for ext in possible_extensions:
                filepath = os.path.join(image_dir, filename + ext)
                if os.path.isfile(filepath):
                    image_paths.append(filepath)
                    found = True
                    break
            if not found:
                print(f"Image file not found for ID: {filename}")
                image_paths.append(None)
        else:
            filepath = os.path.join(image_dir, filename)
            if os.path.isfile(filepath):
                image_paths.append(filepath)
            else:
                print(f"Image file not found: {filepath}")
                image_paths.append(None)
    metadata['ImagePath'] = image_paths
    return metadata

metadata = get_image_paths(metadata, IMAGE_DIR)

# Remove entries with missing images
metadata = metadata[metadata['ImagePath'].notnull()]
metadata_processed = metadata_processed.loc[metadata.index].reset_index(drop=True)
metadata = metadata.reset_index(drop=True)

# Drop diagnostic-related columns from features
diagnostic_cols = ['diagnostic_ACK', 'diagnostic_BCC', 'diagnostic_MEL', 'diagnostic_NEV', 'diagnostic_SCC', 'diagnostic_SEK']
metadata_processed = metadata_processed.drop(columns=diagnostic_cols)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(metadata['diagnostic'])
num_classes = len(label_encoder.classes_)

# Split data into features and labels
X_meta = metadata_processed.reset_index(drop=True)
X_img_paths = metadata['ImagePath'].reset_index(drop=True)
y = pd.Series(y_encoded)

X_train_meta, X_temp_meta, X_train_img_paths, X_temp_img_paths, y_train, y_temp = train_test_split(
    X_meta,
    X_img_paths,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_val_meta, X_test_meta, X_val_img_paths, X_test_img_paths, y_val, y_test = train_test_split(
    X_temp_meta,
    X_temp_img_paths,
    y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp
)

# Load augmented metadata + image paths
aug_meta_df   = pd.read_csv("D:/PAD-UFES/augmented_metadata.csv")
aug_labels_df = pd.read_csv("D:/PAD-UFES/augmented_labels.csv")

# Combine augmented samples with training set
X_train_meta_final = pd.concat([X_train_meta, aug_meta_df], ignore_index=True)
X_train_img_paths_final = pd.concat([X_train_img_paths.reset_index(drop=True),
                                     aug_labels_df['ImagePath']], ignore_index=True)
y_train_final = pd.concat([y_train.reset_index(drop=True),
                           aug_labels_df['Label']], ignore_index=True)

class PADUFESDataset(Dataset):
    def __init__(self, img_paths, meta_data, labels, transform=None):
        self.img_paths = img_paths.reset_index(drop=True)
        self.meta_data = meta_data.reset_index(drop=True)
        self.labels = pd.Series(labels).reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        meta = torch.tensor(self.meta_data.iloc[idx].values.astype(np.float32))
        label = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        return image, meta, label

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),  # Random horizontal flip
    transforms.RandomRotation(70),          
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = PADUFESDataset(X_train_img_paths_final, X_train_meta_final, y_train_final, transform=train_transform)
val_dataset = PADUFESDataset(X_val_img_paths, X_val_meta, y_val, transform=val_test_transform)
test_dataset = PADUFESDataset(X_test_img_paths, X_test_meta, y_test, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print(f"\n✅ Loaded Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_cluster import knn_graph
import timm


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_cluster import knn_graph
import timm


class EarlyFusionWithDynamicGCN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        # === MobileViT Backbone (Original - 3 channels only) ===
        self.mobilevit = timm.create_model("mobilevit_s.cvnets_in1k", pretrained=True, num_classes=0)
        
        # Keep original stem - no modification needed for 3-channel input
        self.mobilevit.stages = nn.Sequential(*list(self.mobilevit.stages.children())[:4])
        self.mobilevit.final_conv = nn.Identity()
        self.mobilevit.head = nn.Identity()

        # === Post Conv ===
        self.post_conv = nn.Sequential(
            nn.Conv2d(128, 160, kernel_size=1, bias=False),
            nn.BatchNorm2d(160),
            nn.ReLU(inplace=True)
        )

        # === Classifier ===
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(160, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, img, meta=None, batch_idx=None):
        # Only use image - ignore metadata and batch_idx
        B = img.size(0)

        # CNN processing
        x_cnn = self.mobilevit.stem(img)  # [B, 3, 224, 224] -> features
        x_cnn = self.mobilevit.stages(x_cnn)
        x_cnn = self.post_conv(x_cnn)
        x_cnn = self.pool(x_cnn).view(B, -1)  # [B, 160]

        return self.classifier(x_cnn)

from torchinfo import summary

input_dim_meta = 59
num_classes = 6
batch_size = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EarlyFusionWithDynamicGCN(num_classes).to(device)

dummy_img = torch.randn(batch_size, 3, 224, 224).to(device)
dummy_meta = torch.randn(batch_size, input_dim_meta).to(device)
dummy_batch_idx = torch.arange(batch_size).to(device)

summary(
    model,
    input_data=[dummy_img, dummy_meta, dummy_batch_idx],
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    depth=3
)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


# =========================================================
# Utility Functions
# =========================================================
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)


def evaluate_model(model, test_loader, device):
    model.eval()
    all_labels, all_preds = [], []

    with torch.no_grad():
        for images, metas, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)  # Only pass image
            preds = outputs.argmax(dim=1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test F1 Score: {f1:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")

    return accuracy, f1, precision, recall


# =========================================================
# NEW: Averaged Training Curve Plotting
# =========================================================
def plot_average_training_curves(all_histories, save_path="averaged_training_curves.png"):
    import numpy as np
    import matplotlib.pyplot as plt

    # ---- Set global bold font ----
    plt.rcParams['font.weight'] = 'bold'
    plt.rcParams['axes.labelweight'] = 'bold'
    plt.rcParams['axes.titleweight'] = 'bold'
    plt.rcParams['xtick.labelsize'] = 12
    plt.rcParams['ytick.labelsize'] = 12

    train_loss = np.array(all_histories["train_loss"])
    val_loss = np.array(all_histories["val_loss"])
    train_acc = np.array(all_histories["train_acc"])
    val_acc = np.array(all_histories["val_acc"])

    epochs = np.arange(1, train_loss.shape[1] + 1)

    plt.figure(figsize=(12, 5))

    # -------- Loss subplot --------
    plt.subplot(1, 2, 1)

    plt.plot(epochs, train_loss.mean(axis=0), label="Train Loss")
    plt.fill_between(
        epochs,
        train_loss.mean(axis=0) - train_loss.std(axis=0),
        train_loss.mean(axis=0) + train_loss.std(axis=0),
        alpha=0.25
    )

    plt.plot(epochs, val_loss.mean(axis=0), label="Validation Loss")
    plt.fill_between(
        epochs,
        val_loss.mean(axis=0) - val_loss.std(axis=0),
        val_loss.mean(axis=0) + val_loss.std(axis=0),
        alpha=0.25
    )

    plt.xlabel("Epochs", fontweight="bold")
    plt.ylabel("Loss", fontweight="bold")
    plt.title("Training and Validation Loss (Averaged Across Runs)", fontweight="bold")
    plt.legend()

    # -------- Accuracy subplot --------
    plt.subplot(1, 2, 2)

    plt.plot(epochs, train_acc.mean(axis=0), label="Train Accuracy")
    plt.fill_between(
        epochs,
        train_acc.mean(axis=0) - train_acc.std(axis=0),
        train_acc.mean(axis=0) + train_acc.std(axis=0),
        alpha=0.25
    )

    plt.plot(epochs, val_acc.mean(axis=0), label="Validation Accuracy")
    plt.fill_between(
        epochs,
        val_acc.mean(axis=0) - val_acc.std(axis=0),
        val_acc.mean(axis=0) + val_acc.std(axis=0),
        alpha=0.25
    )

    plt.xlabel("Epochs", fontweight="bold")
    plt.ylabel("Accuracy", fontweight="bold")
    plt.title("Training and Validation Accuracy (Averaged Across Runs)", fontweight="bold")
    plt.legend()

    plt.tight_layout()

    # ---- SAVE AT 650 DPI ----
    plt.savefig(save_path, dpi=650, bbox_inches="tight")

    plt.show()


def train_model(model, train_loader, val_loader, device, epochs=100, patience=10):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', patience=5, verbose=True)

    best_val_accuracy = 0.0
    best_model_state = None
    patience_counter = 0
    history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}

    for epoch in range(epochs):
        # Training
        model.train()
        train_loss, correct, total = 0.0, 0, 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

        for images, metas, labels in pbar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)  # Only pass image
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_accuracy = correct / total
        history["train_loss"].append(train_loss / len(train_loader))
        history["train_acc"].append(train_accuracy)

        # Validation
        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for images, metas, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)  # Only pass image
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                correct += predicted.eq(labels).sum().item()
                total += labels.size(0)

        val_accuracy = correct / total
        history["val_loss"].append(val_loss / len(val_loader))
        history["val_acc"].append(val_accuracy)

        print(f"Epoch {epoch+1}: Train Loss={train_loss/len(train_loader):.4f}, Train Acc={train_accuracy:.4f}")
        print(f"Epoch {epoch+1}: Val Loss={val_loss/len(val_loader):.4f}, Val Acc={val_accuracy:.4f}")

        # Early Stopping
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

        scheduler.step(val_accuracy)

    return best_model_state, history, best_val_accuracy


# =========================================================
# Main Experiment Loop (MULTI-SEED RUNS)
# =========================================================

seeds = [42, 123]
best_overall_model = None
best_overall_accuracy = 0.0

results = {"accuracy": [], "f1": [], "precision": [], "recall": []}

# NEW: store histories for averaging
all_histories = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}

for seed in seeds:
    print(f"\n--- Training with Seed {seed} ---")
    set_seed(seed)

    model = EarlyFusionWithDynamicGCN(num_classes=6).to(device)

    best_model_state, history, val_acc = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        device=device,
        epochs=100,
        patience=10
    )

    # Store history for averaged curves
    all_histories["train_loss"].append(history["train_loss"])
    all_histories["val_loss"].append(history["val_loss"])
    all_histories["train_acc"].append(history["train_acc"])
    all_histories["val_acc"].append(history["val_acc"])

    # ---- Final Test ----
    model.load_state_dict(best_model_state)
    acc, f1, prec, recall = evaluate_model(model, test_loader, device)

    results["accuracy"].append(acc)
    results["f1"].append(f1)
    results["precision"].append(prec)
    results["recall"].append(recall)

    if val_acc > best_overall_accuracy:
        best_overall_accuracy = val_acc
        best_overall_model = model

# ---- Save Best Model ----
torch.save(best_overall_model.state_dict(), "ImageOnly_Model.pth")
print(f"\nBest Val Accuracy Model Saved (Acc={best_overall_accuracy:.4f})")

# ---- Summary ----
print("\n--- Final Evaluation Across Seeds ---")
for metric in results:
    print(f"{metric.capitalize()}: {np.mean(results[metric]):.4f} ± {np.std(results[metric]):.4f}")

# ---- PLOT AVERAGED TRAINING CURVES ----

In [ ]:
def plot_average_training_curves(all_histories, save_path="averaged_training_curves.png"):
    import numpy as np
    import matplotlib.pyplot as plt

    # Bold fonts
    plt.rcParams['font.weight'] = 'bold'
    plt.rcParams['axes.labelweight'] = 'bold'
    plt.rcParams['axes.titleweight'] = 'bold'
    plt.rcParams['xtick.labelsize'] = 12
    plt.rcParams['ytick.labelsize'] = 12

    # ---- Find max epoch length across all runs ----
    max_epochs = max(len(h) for h in all_histories["train_loss"])

    # ---- Padding helper ----
    def pad_list(l):
        return l + [np.nan] * (max_epochs - len(l))

    # ---- Convert histories to padded arrays ----
    train_loss = np.array([pad_list(h) for h in all_histories["train_loss"]])
    val_loss   = np.array([pad_list(h) for h in all_histories["val_loss"]])
    train_acc  = np.array([pad_list(h) for h in all_histories["train_acc"]])
    val_acc    = np.array([pad_list(h) for h in all_histories["val_acc"]])

    epochs = np.arange(1, max_epochs + 1)

    plt.figure(figsize=(12, 5))

    # ===== Loss subplot =====
    plt.subplot(1, 2, 1)
    plt.plot(epochs, np.nanmean(train_loss, axis=0), label="Train Loss")
    plt.fill_between(epochs,
                     np.nanmean(train_loss, axis=0) - np.nanstd(train_loss, axis=0),
                     np.nanmean(train_loss, axis=0) + np.nanstd(train_loss, axis=0),
                     alpha=0.25)
    plt.plot(epochs, np.nanmean(val_loss, axis=0), label="Validation Loss")
    plt.fill_between(epochs,
                     np.nanmean(val_loss, axis=0) - np.nanstd(val_loss, axis=0),
                     np.nanmean(val_loss, axis=0) + np.nanstd(val_loss, axis=0),
                     alpha=0.25)

    plt.xlabel("Epochs", fontweight="bold")
    plt.ylabel("Loss", fontweight="bold")
    plt.title("Training and Validation Loss (Averaged Across Runs)", fontweight="bold")
    plt.legend()

    # ===== Accuracy subplot =====
    plt.subplot(1, 2, 2)
    plt.plot(epochs, np.nanmean(train_acc, axis=0), label="Train Accuracy")
    plt.fill_between(epochs,
                     np.nanmean(train_acc, axis=0) - np.nanstd(train_acc, axis=0),
                     np.nanmean(train_acc, axis=0) + np.nanstd(train_acc, axis=0),
                     alpha=0.25)
    plt.plot(epochs, np.nanmean(val_acc, axis=0), label="Validation Accuracy")
    plt.fill_between(epochs,
                     np.nanmean(val_acc, axis=0) - np.nanstd(val_acc, axis=0),
                     np.nanmean(val_acc, axis=0) + np.nanstd(val_acc, axis=0),
                     alpha=0.25)

    plt.xlabel("Epochs", fontweight="bold")
    plt.ylabel("Accuracy", fontweight="bold")
    plt.title("Training and Validation Accuracy (Averaged Across Runs)", fontweight="bold")
    plt.legend()

    plt.tight_layout()
    plt.show()


plot_average_training_curves(all_histories)

In [ ]:
import torch
import torch.nn as nn
import time
import numpy as np
from ptflops import get_model_complexity_info


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

params_m = count_parameters(model) / 1e6
print(f"Total Trainable Parameters: {params_m:.3f} M")


# For image-only model, no wrapper needed - direct FLOPs measurement
with torch.no_grad():
    flops, params = get_model_complexity_info(
        model,
        (3, 224, 224),          # image only input
        as_strings=False,
        print_per_layer_stat=False,
        verbose=False
    )

flops_g = flops / 1e9
print(f"FLOPs: {flops_g:.3f} GFLOPs")


def measure_gpu_latency(model, device, runs=200, warmup=30):
    if not torch.cuda.is_available():
        print("CUDA not available, skipping GPU latency.")
        return None, None, None

    model.eval()
    model.to(device)

    # Fixed dummy input - image only
    dummy_img = torch.randn(1, 3, 224, 224, device=device)

    # Warm-up
    for _ in range(warmup):
        _ = model(dummy_img)
    torch.cuda.synchronize()

    times = []

    start_event = torch.cuda.Event(enable_timing=True)
    end_event = torch.cuda.Event(enable_timing=True)

    for _ in range(runs):
        start_event.record()
        _ = model(dummy_img)
        end_event.record()
        torch.cuda.synchronize()
        elapsed_ms = start_event.elapsed_time(end_event)  # ms
        times.append(elapsed_ms)

    times = np.array(times)
    mean = times.mean()
    std = times.std()
    fps = 1000.0 / mean
    return mean, std, fps

gpu_mean, gpu_std, gpu_fps = measure_gpu_latency(model, device)

if gpu_mean is not None:
    print(f"GPU Latency: {gpu_mean:.3f} ± {gpu_std:.3f} ms")
    print(f"GPU FPS: {gpu_fps:.2f}")
else:
    print("GPU metrics not computed (no CUDA).")


def measure_cpu_latency(model, runs=100, warmup=20):
    model_cpu = model.cpu()
    model_cpu.eval()

    dummy_img = torch.randn(1, 3, 224, 224)

    # Warm-up
    for _ in range(warmup):
        _ = model_cpu(dummy_img)

    times = []
    for _ in range(runs):
        start = time.perf_counter()
        _ = model_cpu(dummy_img)
        end = time.perf_counter()
        times.append((end - start) * 1000.0)  # ms

    times = np.array(times)
    mean = times.mean()
    std = times.std()
    fps = 1000.0 / mean
    return mean, std, fps

cpu_mean, cpu_std, cpu_fps = measure_cpu_latency(model)
print(f"CPU Latency: {cpu_mean:.3f} ± {cpu_std:.3f} ms")
print(f"CPU FPS: {cpu_fps:.2f}")



efficiency_stats = {
    "params_M": params_m,
    "flops_G": flops_g,
    "gpu_latency_ms_mean": gpu_mean,
    "gpu_latency_ms_std": gpu_std,
    "gpu_fps": gpu_fps,
    "cpu_latency_ms_mean": cpu_mean,
    "cpu_latency_ms_std": cpu_std,
    "cpu_fps": cpu_fps,
}

print("\nEfficiency stats dict (for your table):")
for k, v in efficiency_stats.items():
    print(f"{k}: {v}")


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
import torch.nn.functional as F

# Load the best model for evaluation
best_model = EarlyFusionWithDynamicGCN(num_classes=6).to(device)
best_model.load_state_dict(torch.load("ImageOnly_Model.pth"))
best_model.eval()

# Collect true labels and predictions
all_labels = []
all_preds = []
all_probs = []

with torch.no_grad():
    for images, metas, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)  # Only pass image
        probs = F.softmax(outputs, dim=1)
        preds = probs.argmax(dim=1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)
all_probs = np.array(all_probs)

# Get class names from label encoder
class_names = label_encoder.classes_

# Compute classification report
class_report = classification_report(all_labels, all_preds, target_names=class_names, digits=4)

# Compute normalized confusion matrix
conf_matrix = confusion_matrix(all_labels, all_preds, normalize="true")

# Display classification report
print("\nClassification Report:\n")
print(class_report)

# Display confusion matrix (Blues colormap)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt=".2f", xticklabels=class_names, yticklabels=class_names, cbar=True)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Normalized Confusion Matrix")
plt.show()

# Compute and plot ROC-AUC curve for each class
plt.figure(figsize=(10, 6))

for i, class_name in enumerate(class_names):
    fpr, tpr, _ = roc_curve((all_labels == i).astype(int), all_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{class_name} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], "k--")  # Diagonal line for reference
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC-AUC Curve")
plt.legend(loc="lower right")
plt.show()

# Compute and plot Precision-Recall Curve
plt.figure(figsize=(10, 6))

for i, class_name in enumerate(class_names):
    precision, recall, _ = precision_recall_curve((all_labels == i).astype(int), all_probs[:, i])
    plt.plot(recall, precision, label=f"{class_name}")

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.show()
